In [1]:
import time
from create_vectorDB import VECTORDB
from create_documents import DOCUMENTS

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [2]:
vec_obj = VECTORDB()
doc_obj = DOCUMENTS()
vec_dir = vec_obj.db_dir

# Read the Ollama embeddings
embeddings = OllamaEmbeddings(model="llama3.1", show_progress=False)
db = Chroma(persist_directory=vec_dir,
            embedding_function=embeddings)

# Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

c:\Users\vinay\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [3]:
# Create Ollama language model - Llama3.1
local_llm = 'llama3.1'

llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=2048,  
                 temperature=0)

In [4]:
# Create prompt template
template = """<bos><start_of_turn>user\nYou are a chatbot built by ctruh. \
ctruh has an app/website/tool/platform that can create virtual environments. The context has all the required details on the same \
Your sole responsibility is to receive user feedback and respond back appropriately. \
Do not answer for 'how', 'why', 'what', 'is', 'when' kind of questions, respond back saying you can only receive feedback, and reach out to the support team for answering queries. \
Format the answers appropriately and be enthusiastic and empathetic while responding back to the feedback. \
Respond with full sentences with correct spellings and right punctuations. \
To help you on how to respond back to the user feedback, the context has some feedback-response samples \
that look like the following - \
Feedback: (this contains the sample user feedback) \
Response: (this contains the sample response for the user feedback) \
Use these ONLY as references for responding back to the user feedback. \
Always answer succinctly, do not give any additional information to the user other than responding back to the feedback. \

CONTEXT: {context}

PROMPT: {prompt}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [5]:
# Create the RAG chain using LCEL with prompt printing and streaming output
rag_chain = (
    {"context": retriever, "prompt": RunnablePassthrough()}
    | prompt
    | llm
)

In [6]:
# Function to send prompt
def send_prompt(prompt):
    print("User:", prompt)
    print("Cbot:", end=" ", flush=True)
    st = time.time()
    for chunk in rag_chain.stream(prompt):
        print(chunk.content, end="", flush=True)
    print("\n\t({}) sec\n\n".format(time.time() - st))

In [7]:
while True:
    user_prompt = input("Send a message (or type 'quit' to exit): ")
    if user_prompt.lower() == 'quit':
        break
    answer = send_prompt(user_prompt)
    # print("\nFull answer received.\n")

User: hi
Cbot: Hello! I'm here to receive your feedback. Please feel free to share any thoughts or concerns you may have, and I'll do my best to respond accordingly. If you need more information or clarification on something, please don't hesitate to reach out to the support team for assistance. What's on your mind?
	(5.9676690101623535) sec


User: your UI is just the best! its so sleek and simple to use, i did not have to put any effort
Cbot: Thank you for your wonderful feedback about our UI! We're thrilled that it's been a pleasure to use for you. Your kind words are much appreciated and will be shared with the team! Would you like to share any other thoughts or suggestions?
	(6.439154386520386) sec


User: the updates are so fast, every update i see new asset added. This is a huge value add, i am always looking for new updates!
Cbot: We're thrilled to hear that you appreciate the frequent updates and new assets being added! Your enthusiasm motivates us to keep delivering value and